Results from demographic + experience smartwatch survey
====

In [ ]:
%load_ext autoreload
%autoreload 2

In [12]:
"""
Read in the questionnaire data

"""
import pandas as pd

# Need to mount the SEACO RDSF drive here
path = "/mnt/z/SEACO data/SEACO-CH20 qnaire data/SEACO_CH20_17082022_de_id.csv"
qnaire_data = pd.read_csv(path)

qnaire_data.head()

/tmp/ipykernel_10437/3104238101.py:9: DtypeWarning: Columns (17,25,26,52,197,198,200,201,209,210,212,213,216,218,219,222,225,227,228,230,231,233,234,236,237,239,240,243,246,248,249,252,254,255,257,258,261,263,264,267,269,270,272,273,276,279,281,284,291,293,294,296,297,300,302,303,305,306,309,311,312,314,315,317,318,321,324,327,330,344,351,352,354,355,360,361,362,363,364,369,370,371,372,373,378,379,380,381,382,387,390,391,396,397,398,399,400,406,407,415,425,432,434,435,436,446,456,486,499,504,506,512,513,549) have mixed types. Specify dtype option on import or set low_memory=False.
  qnaire_data = pd.read_csv(path)


,deviceid,updatelistselected,barcodehouse,residents_id,respondent_status,verify_note,verify_address,address_barcode,respondent_sex,respondent_ethnicity,...,field_note,formdef_version,key,review_quality,review_corrections,submissiondate,starttime,endtime,dob,bp_start
0,1111dda705a8df02,2104,JBI000024,2104,1,NaN,1.0,1.0,1.0,1.0,...,Done blood,2.112000e+09,uuid:9c4dfbbd-efd6-4dfa-bd94-35e1e64a7f45,OKAY,"nurul.syazwana@monash.edu (Mar 11, 2022 4:21:3...",19jan2022 09:17:38,18jan2022 21:44:56,19jan2022 08:59:50,13mar2007,18jan2022
1,1833a79c47db6bdc,2864,GMH000071,2864,1,NaN,1.0,1.0,1.0,1.0,...,NaN,2.112000e+09,uuid:cd20720f-3c19-48f9-bc27-76f749e257c9,OKAY,NaN,26jan2022 12:01:19,21jan2022 16:22:03,24jan2022 11:32:27,11jul2013,24jan2022
2,1833a79c47db6bdc,15697,GMH000980,15697,1,NaN,1.0,1.0,1.0,1.0,...,NaN,2.112000e+09,uuid:39a85c45-a319-45e7-bbab-ff683d328ef9,GOOD,NaN,26jan2022 12:01:19,24jan2022 11:37:03,24jan2022 12:01:56,10mar2008,24jan2022
3,1833a79c47db6bdc,1382,JBI000014,1382,1,NaN,1.0,1.0,1.0,1.0,...,NaN,2.112000e+09,uuid:3471f3f6-fe16-4d76-b747-210b12d2266a,GOOD,NaN,26jan2022 12:01:20,24jan2022 12:12:45,26jan2022 11:35:36,13jan2013,26jan2022
4,1833a79c47db6bdc,3632,JBI000329,3632,1,NaN,1.0,1.0,2.0,1.0,...,NaN,2.112000e+09,uuid:42cc75b5-cd1e-4099-b8a7-f92963922c56,OKAY,NaN,28jan2022 11:20:00,06jan2022 22:38:14,26jan2022 13:44:00,29mar2013,07jan2022


In [16]:
"""
Choose only respondents who agreed to take part

"""
accepted_df = qnaire_data[qnaire_data["respondent_status"] == 1].copy()
len(accepted_df)

626

In [18]:
"""
Add a column for age group

"""

lims = (7, 9, 18)
accepted_df["child"] = (accepted_df["age_dob"] >= lims[0]) & (
    accepted_df["age_dob"] <= lims[1]
)
accepted_df["adolescent"] = (accepted_df["age_dob"] > lims[1]) & (
    accepted_df["age_dob"] <= lims[2]
)

# Check no overlap
assert not (accepted_df["child"] & accepted_df["adolescent"]).any()

# Check no unassigned
assert (accepted_df["child"] | accepted_df["adolescent"]).all()

respondent_status
1    626
Name: count, dtype: int64